In [2]:
#Imports relevantes 
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pickle

In [ ]:
# Función para entrenar el modelo
def entrenar_modelo_palabras_clave(data):
    """
    Args:
    - data : Un DataFrame con las siguientes columnas: ['titulo', 'localidad', 'palabras_clave', 'categoria']

    Returns:
    - modelo_entrenado: El modelo entrenado que se puede utilizar para clasificar nuevos textos.
    - vectorizer: El vectorizador que se utilizará para transformar los nuevos textos.
    """
    
    # Crear una columna combinada de título, localidad y palabras clave
    data['combined_features'] = data['titulo'] + ' ' + data['localidad'] + ' ' + data['palabras_clave'].apply(lambda x: ' '.join(x))
    
    # Vectorización de los textos
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(data['combined_features'])
    y = data['categoria']
    
    # Dividir los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Entrenar el modelo
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    
    # Evaluar el modelo
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Exactitud del modelo: {accuracy * 100:.2f}%')
    
    return clf, vectorizer

def entrenar_modelo_texto_completo(data):
    """
    Args:
    - data : Un DataFrame con las siguientes columnas: ['titulo', 'localidad', 'texto', 'categoria']

    Returns:
    - modelo_entrenado: El modelo entrenado que se puede utilizar para clasificar nuevos textos.
    - vectorizer: El vectorizador que se utilizará para transformar los nuevos textos.
    """
    
    # Crear una columna combinada de título, localidad y descripción
    data['combined_features'] = data['titulo'] + ' ' + data['localidad'] + ' ' + data['texto']
    
    # Vectorización de los textos
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(data['combined_features'])
    y = data['categoria']
    
    # Dividir los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Entrenar el modelo
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    
    # Evaluar el modelo
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Exactitud del modelo: {accuracy * 100:.2f}%')
    
    return clf, vectorizer

In [ ]:
# Función para clasificar un nuevo texto
def clasificar_texto(modelo_entrenado, vectorizer, titulo, localidad, palabras_clave):
    """
    Args:
    - modelo_entrenado: El modelo entrenado previamente con `entrenar_modelo`.
    - vectorizer: El vectorizador entrenado previamente con `entrenar_modelo`.
    - titulo (str): El título del nuevo texto.
    - localidad (str): La localidad asociada al texto.
    - palabras_clave (list): Lista de palabras clave asociadas al texto.

    Returns:
    - categoria_predicha (str): La categoría predicha para el nuevo texto.
    """
    
    # Combinar el título, localidad y palabras clave en un solo string
    new_combined = titulo + ' ' + localidad + ' ' + ' '.join(palabras_clave)
    
    # Vectorizar el nuevo texto
    new_combined_vectorized = vectorizer.transform([new_combined])
    
    # Predecir la categoría
    predicted_category = modelo_entrenado.predict(new_combined_vectorized)[0]
    
    return predicted_category